<a href="https://colab.research.google.com/github/ozgunguler/Software-Defect-Detection-System-with-Promise-Dataset/blob/main/CENG480_ipynb_%C3%B6zg%C3%BCn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing

In [ ]:
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 5)

from sklearn.ensemble import BaggingClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import AdaBoostClassifier


from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc, confusion_matrix, f1_score

from sklearn.preprocessing import LabelBinarizer


from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.dummy import DummyClassifier
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture


In [ ]:
df={}
df[0] = pd.read_csv("/content/drive/MyDrive/dataset/antV2/5.csv")
df[1] = pd.read_csv("/content/drive/MyDrive/dataset/camelV1/4.csv")
df[2] = pd.read_csv("/content/drive/MyDrive/dataset/ivy/3.csv")
df[3] = pd.read_csv("/content/drive/MyDrive/dataset/jeditV2/5.csv")
df[4] = pd.read_csv("/content/drive/MyDrive/dataset/log4j/3.csv")
df[5] = pd.read_csv("/content/drive/MyDrive/dataset/lucene/3.csv")
df[6] = pd.read_csv("/content/drive/MyDrive/dataset/poiV1/4.csv")
df[7] = pd.read_csv("/content/drive/MyDrive/dataset/synapse/3.csv")
df[8]  = pd.read_csv("/content/drive/MyDrive/dataset/velocity/3.csv")
df[9] = pd.read_csv("/content/drive/MyDrive/dataset/xercesV1/4.csv")
for i in range(10):
  df[i] = df[i].drop(columns=['name','version','name.1'])

# Visualizing data

Let's see every label in histogram graphs and make analysis for what we will do in next steps

In [ ]:
for i in range(10):
  for col in df[i].columns:
    plt.hist(df[i][col],rwidth=0.9,alpha=0.3,color='blue',bins=20,edgecolor='red')
    plt.title(col)
    plt.show()
  print(i)

As we see

# Detecting and treating outliers

In [ ]:
for i in range(10):
  for col in (df[i].columns):
    if col == 'bug':
      continue
    q1=df[i][col].quantile(0.25)
    q3=df[i][col].quantile(0.75)
    IQR=q3-q1
    lowLim = q1 - 1.5 * IQR
    upLim = q3 + 1.5 * IQR
    df[i][col][ (df[i][col] > upLim) | (df[i][col] < lowLim)] = df[i][col].mean()

# Shapes of datasets after treating outliers

In [ ]:
for i in range(10):
  print(df[i].shape)

In [ ]:
print(df[1]['bug'])

# İlk Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
dfantV2 = df[0].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = dfantV2.iloc[:, :-1]
y = dfantV2["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
dfantV2.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (521, 20) (521,) Test: (112, 20) (112,) Cross Validation (112, 20) (112,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (521, 20) (521,) Test: (112, 20) (112,) Cross Validation (112, 20) (112,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 1.0


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'lcom', 'loc','amc', 'cam', 'ce','wmc','cbo','avg_cc','lcom3','max_cc','Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'lcom', 'loc','amc', 'cam', 'ce','wmc','cbo','avg_cc','lcom3','max_cc']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);



In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
         
        n_classes = 1 #(0, 1)
        if multi:
            n_classes = 3

        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['rfc', 'lcom', 'loc','amc', 'cam', 'ce','wmc','cbo','avg_cc','lcom3','max_cc']]
X_test_scaled_featured = X_test_scaled[['rfc', 'lcom', 'loc','amc', 'cam', 'ce','wmc','cbo','avg_cc','lcom3','max_cc']]
X_cv_featured = X_cv[['rfc', 'lcom', 'loc','amc', 'cam', 'ce','wmc','cbo','avg_cc','lcom3','max_cc']]


In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured , y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier,X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 20, 'max_features': 0.4, 'n_estimators': 500}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 0.1, 'n_estimators': 100}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# İkinci Dataset


In [ ]:
# Shuffle data before removing classname to keep mapping
camelV1 = df[1].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = camelV1.iloc[:, :-1]
y = camelV1["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
camelV1.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (675, 20) (675,) Test: (145, 20) (145,) Cross Validation (145, 20) (145,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 0.9955555555555555


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['lcom3', 'cbo', 'loc','amc', 'ca', 'avg_cc','cam','ce','rfc','mfa','npm', 'Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['lcom3', 'cbo', 'loc','amc', 'ca', 'avg_cc','cam','ce','rfc','mfa','npm']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);



In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['lcom3', 'cbo', 'loc','amc', 'ca', 'avg_cc','cam','ce','rfc','mfa','npm']]
X_test_scaled_featured = X_test_scaled[['lcom3', 'cbo', 'loc','amc', 'ca', 'avg_cc','cam','ce','rfc','mfa','npm']]
X_cv_featured = X_cv[['lcom3', 'cbo', 'loc','amc', 'ca', 'avg_cc','cam','ce','rfc','mfa','npm']]

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 20, 'max_features': 0.4, 'n_estimators': 500}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi = False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 1.0, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# 3. Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
ivy = df[2].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = ivy.iloc[:, :-1]
y = ivy["bug"]

print("X:", X.shape)
print("y:", y.shape)

X: (352, 20)
y: (352,)


In [ ]:
ivy.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (246, 20) (246,) Test: (53, 20) (53,) Cross Validation (53, 20) (53,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 1.0


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'lcom3', 'loc','amc', 'cam','npm', 'cbo','lcom','avg_cc','max_cc','ca','Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'lcom3', 'loc','amc', 'cam','npm', 'cbo','lcom','avg_cc','max_cc','ca']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);



In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['rfc', 'lcom3', 'loc','amc', 'cam','npm', 'cbo','lcom','avg_cc','max_cc','ca']]
X_test_scaled_featured = X_test_scaled[['rfc', 'lcom3', 'loc','amc', 'cam','npm', 'cbo','lcom','avg_cc','max_cc','ca']]
X_cv_featured = X_cv[['rfc', 'lcom3', 'loc','amc', 'cam','npm', 'cbo','lcom','avg_cc','max_cc','ca']]


In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE(k_neighbors=3).fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured , y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier,X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'manhattan', 'n_neighbors': 17, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 10, 'max_features': 0.4, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured , y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 0.5, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured , y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

,Accuracy,ROC,F1-Score
K-Nearest Neighbor | Binary,0.773585,0.819444,0.708791
Random Forest | Binary,0.811321,0.577020,0.588509
AdaBoost Classifier | Binary,0.754717,0.542929,0.544613
Support-Vector Machine | Binary,0.830189,0.500000,0.453608


# Dördüncü Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
jeditV2 = df[3].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = jeditV2.iloc[:, :-1]
y = jeditV2["bug"]

print("X:", X.shape)
print("y:", y.shape)

X: (492, 20)
y: (492,)


In [ ]:
jeditV2.describe()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
count,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000,492.000000
mean,8.239747,2.135832,0.056349,9.817916,30.825996,50.245910,4.387137,5.522011,4.876673,1.087181,246.647226,0.496919,0.483467,0.415344,0.462153,0.384217,0.487433,24.088348,4.670674,1.542383,0.024390
std,6.097572,1.602042,0.148547,6.747350,26.853251,90.815358,3.868115,4.654132,3.671725,0.651449,236.152349,0.464965,0.624021,0.440211,0.254189,0.580561,0.684505,18.844989,4.236952,0.944146,0.167084
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,0.000000,4.000000,9.000000,1.000000,1.000000,2.000000,2.000000,0.666071,41.750000,0.000000,0.000000,0.000000,0.259815,0.000000,0.000000,8.333333,1.000000,0.851150,0.000000
50%,7.000000,1.500000,0.000000,8.500000,24.500000,6.000000,3.000000,5.000000,4.000000,0.846543,176.000000,0.500000,0.000000,0.052632,0.425735,0.000000,0.000000,21.500000,3.000000,1.333300,0.000000
75%,12.351626,2.367886,0.000000,14.317073,44.000000,36.000000,7.000000,8.000000,7.851626,2.000000,411.306911,1.000000,1.000000,0.933333,0.625000,1.000000,1.000000,34.728807,6.587398,2.000000,0.000000
max,27.000000,6.000000,0.447154,31.000000,117.000000,259.906504,16.000000,19.000000,17.000000,2.000000,1034.000000,1.000000,2.000000,1.000000,1.000000,2.000000,2.000000,80.250000,18.000000,4.368400,2.000000


In [ ]:
X.head()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
394,5.0,2.367886,0.000000,7.000000,34.0,0.000000,3.000000,7.0,3.000000,0.666667,159.0,1.0,1.0,0.995043,0.320000,0.595528,1.497967,30.200000,4.000000,1.600000
163,10.0,2.367886,0.000000,10.000000,69.0,19.000000,4.000000,10.0,3.000000,0.740741,403.0,1.0,2.0,0.986466,0.260000,0.000000,0.000000,38.700000,4.000000,1.700000
357,20.0,1.000000,0.000000,14.317073,45.0,259.906504,8.739837,0.0,7.851626,2.000000,841.0,0.0,0.0,0.000000,0.187500,0.000000,0.000000,41.050000,6.587398,1.827174
226,5.0,1.000000,0.447154,7.000000,20.0,4.000000,3.000000,5.0,5.000000,0.500000,126.0,1.0,0.0,0.000000,0.440000,0.000000,0.000000,23.800000,7.000000,2.000000
77,6.0,2.000000,0.000000,14.000000,16.0,15.000000,3.000000,11.0,2.000000,1.000000,113.0,0.0,1.0,0.772727,0.466667,0.000000,0.000000,17.666667,2.000000,1.000000


In [ ]:
y.head()

0    0
1    0
2    0
3    0
4    0
dtype: int64

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y == 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (344, 20) (344,) Test: (74, 20) (74,) Cross Validation (74, 20) (74,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (344, 20) (344,) Test: (74, 20) (74,) Cross Validation (74, 20) (74,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 0.9941860465116279


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'ce', 'loc','amc', 'avg_cc', 'cam','lcom3','cbo','ca','dit','max_cc','cbm','Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'ce', 'loc','amc', 'avg_cc', 'cam','lcom3','cbo','ca','dit','max_cc','cbm']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);



In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['rfc', 'ce', 'loc','amc', 'avg_cc', 'cam','lcom3','cbo','ca','dit','max_cc','cbm']]
X_test_scaled_featured = X_test_scaled[['rfc', 'ce', 'loc','amc', 'avg_cc', 'cam','lcom3','cbo','ca','dit','max_cc','cbm']]
X_cv_featured = X_cv[['rfc', 'ce', 'loc','amc', 'avg_cc', 'cam','lcom3','cbo','ca','dit','max_cc','cbm']]

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier,X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# Beşinci Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
log4j = df[4].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = log4j.iloc[:, :-1]
y = log4j["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
log4j.describe()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,6.993504,1.490589,0.025699,5.404759,21.463510,14.711624,2.090327,3.262011,4.862558,1.024772,142.500345,0.317335,0.521190,0.326962,0.403621,0.338299,0.317240,17.965703,3.073004,1.176041,2.429268
std,4.809768,0.650872,0.078352,3.735764,14.858741,18.998060,2.255551,2.818555,4.099388,0.610619,122.321117,0.415972,0.646542,0.410018,0.179310,0.584363,0.578585,13.393657,2.312814,0.554181,1.521455
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,1.000000,0.000000,3.000000,10.000000,1.000000,0.000000,1.000000,2.000000,0.666667,41.000000,0.000000,0.000000,0.000000,0.277778,0.000000,0.000000,8.000000,1.000000,0.800000,2.000000
50%,6.000000,1.000000,0.000000,5.000000,18.000000,6.000000,1.000000,3.000000,3.000000,0.833333,123.000000,0.000000,0.000000,0.000000,0.357143,0.000000,0.000000,15.500000,2.000000,1.000000,2.000000
75%,9.000000,2.000000,0.000000,7.609756,31.000000,21.000000,4.000000,5.000000,7.000000,1.500000,200.000000,0.750000,1.000000,0.764706,0.500000,1.000000,0.590244,26.000000,4.000000,1.500000,3.000000
max,20.000000,3.000000,0.263415,17.000000,66.000000,53.931707,10.000000,11.000000,17.000000,2.000000,530.000000,1.000000,2.000000,1.000000,0.900000,2.000000,2.000000,56.692308,11.000000,2.800000,10.000000


In [ ]:
X.head()

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
155,4.0,1.0,0.0,1.000000,11.0,2.000000,1.000000,0.0,2.0,0.777778,61.0,0.0,0.0,0.000000,0.500000,1.0,1.0,13.500000,6.0,2.0000
133,8.0,2.0,0.0,0.000000,10.0,28.000000,0.000000,0.0,8.0,2.000000,21.0,0.0,0.0,0.650000,0.468750,1.0,2.0,1.625000,1.0,0.8750
75,10.0,2.0,0.0,2.000000,37.0,23.000000,0.000000,2.0,7.0,0.833333,301.0,0.0,1.0,0.884058,0.200000,0.0,0.0,28.100000,4.0,1.7000
74,15.0,1.0,0.0,7.609756,15.0,53.931707,4.326829,6.0,15.0,2.000000,15.0,0.0,0.0,0.000000,0.216667,0.0,0.0,0.000000,1.0,1.0000
197,13.0,2.0,0.0,5.000000,35.0,48.000000,0.000000,5.0,11.0,0.791667,259.0,0.0,0.0,0.600000,0.292308,1.0,2.0,18.461538,5.0,1.5385


In [ ]:
y.head()

155    3
133    2
75     1
74     3
197    3
Name: bug, dtype: int64

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

[0      False
1      False
2      False
3      False
4      False
       ...  
200    False
201    False
202    False
203    False
204    False
Length: 205, dtype: bool]


In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (143, 20) (143,) Test: (31, 20) (31,) Cross Validation (31, 20) (31,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (143, 20) (143,) Test: (31, 20) (31,) Cross Validation (31, 20) (31,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['ce', 'avg_cc', 'loc','amc', 'cam', 'cbo', 'mfa', 'npm', 'rfc', 'lcom3','wmc','Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['ce', 'avg_cc', 'loc','amc', 'cam', 'cbo', 'mfa', 'npm', 'rfc', 'lcom3','wmc']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);



In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:

X_train_scaled_featured = X_train_scaled[['ce', 'avg_cc', 'loc','amc', 'cam', 'cbo', 'mfa', 'npm', 'rfc', 'lcom3','wmc']]
X_test_scaled_featured = X_test_scaled[['ce', 'avg_cc', 'loc','amc', 'cam', 'cbo', 'mfa', 'npm', 'rfc', 'lcom3','wmc']]
X_cv_featured = X_cv[['ce', 'avg_cc', 'loc','amc', 'cam', 'cbo', 'mfa', 'npm', 'rfc', 'lcom3','wmc']]


In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE(k_neighbors=1).fit_resample(X_cv_featured, y_cv)



In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# print("Train:", X_train_over_sampled.shape, y_train_over_sampled.shape,
#       "Test:", X_test.shape, y_test.shape,
#       "Cross Validation", X_cv_over_sampled.shape, y_cv_over_sampled.shape)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC,  X_train_over_sampled, y_train_over_sampled,  X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier,X_train_over_sampled, y_train_over_sampled,  X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier,X_train_over_sampled, y_train_over_sampled,  X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 10, 'max_features': 0.4, 'n_estimators': 500}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled,  X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 0.1, 'n_estimators': 100}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# Altıncı Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
lucene = df[5].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = lucene.iloc[:, :-1]
y = lucene["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
lucene.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (238, 20) (238,) Test: (51, 20) (51,) Cross Validation (51, 20) (51,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (238, 20) (238,) Test: (51, 20) (51,) Cross Validation (51, 20) (51,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 1.0


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'cbo', 'loc','amc', 'cam', 'npm', 'ce','avg_cc','lcom3', 'wmc','ca', 'Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'cbo', 'loc','amc', 'cam','npm', 'ce', 'avg_cc','lcom3', 'wmc','ca']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);

In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:

X_train_scaled_featured = X_train_scaled[['rfc', 'cbo', 'loc','amc', 'cam','npm', 'ce', 'avg_cc','lcom3', 'wmc','ca']]
X_test_scaled_featured = X_test_scaled[['rfc', 'cbo', 'loc','amc', 'cam','npm', 'ce', 'avg_cc','lcom3', 'wmc','ca']]
X_cv_featured = X_cv[['rfc', 'cbo', 'loc','amc', 'cam','npm', 'ce', 'avg_cc','lcom3', 'wmc','ca']]

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier,X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 50, 'max_features': 0.4, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 0.5, 'n_estimators': 100}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# Yedinci Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
#Shuffle data before removing classname to keep mapping
poiV1 = df[6].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = poiV1.iloc[:, :-1]
y = poiV1["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
poiV1.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (309, 20) (309,) Test: (67, 20) (67,) Cross Validation (66, 20) (66,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (309, 20) (309,) Test: (67, 20) (67,) Cross Validation (66, 20) (66,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 0.9935275080906149


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'cbo', 'loc','amc', 'ce', 'lcom3', 'lcom', 'max_cc', 'avg_cc', 'npm', 'wmc', 'Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'cbo', 'loc','amc', 'ce', 'lcom3', 'lcom', 'max_cc', 'avg_cc', 'npm', 'wmc']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);

In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['rfc', 'cbo', 'loc','amc', 'ce', 'lcom3', 'lcom', 'max_cc', 'avg_cc', 'npm', 'wmc']]
X_test_scaled_featured = X_test_scaled[['rfc', 'cbo', 'loc','amc', 'ce', 'lcom3', 'lcom', 'max_cc', 'avg_cc', 'npm', 'wmc']]
X_cv_featured = X_cv[['rfc', 'cbo', 'loc','amc', 'ce', 'lcom3', 'lcom', 'max_cc', 'avg_cc', 'npm', 'wmc']]


In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'euclidean', 'n_neighbors': 5, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 10, 'max_features': 1.0, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 0.1, 'n_estimators': 100}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# Sekizinci Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
synapse = df[7].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = synapse.iloc[:, :-1]
y = synapse["bug"]

print("X:", X.shape)
print("y:", y.shape)

X: (256, 20)
y: (256,)


In [ ]:
synapse.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (179, 20) (179,) Test: (39, 20) (39,) Cross Validation (38, 20) (38,)


In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (179, 20) (179,) Test: (39, 20) (39,) Cross Validation (38, 20) (38,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'wmc', 'loc','amc', 'cam', 'avg_cc','ce','max_cc','mfa','ca','lcom3', 'Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'wmc', 'loc','amc', 'cam', 'avg_cc','ce','max_cc','mfa','ca','lcom3']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);

In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['rfc', 'wmc', 'loc','amc', 'cam', 'avg_cc','ce','max_cc','mfa','ca','lcom3']]
X_test_scaled_featured = X_test_scaled[['rfc', 'wmc', 'loc','amc', 'cam', 'avg_cc','ce','max_cc','mfa','ca','lcom3']]
X_cv_featured = X_cv[['rfc', 'wmc', 'loc','amc', 'cam', 'avg_cc','ce','max_cc','mfa','ca','lcom3']]

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 1, 'gamma': 1, 'kernel': 'rbf'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'manhattan', 'n_neighbors': 17, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier,  X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 20, 'max_features': 0.4, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier,  X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

#Dokuzuncu İlk Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
velocity = df[8].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = velocity.iloc[:, :-1]
y = velocity["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
velocity.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 1.0


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['rfc', 'cbo', 'loc','amc', 'cam','ce','npm', 'wmc', 'avg_cc', 'lcom', 'ca', 'Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['rfc', 'cbo', 'loc','amc', 'cam','ce','npm', 'wmc', 'avg_cc', 'lcom', 'ca']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);

In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['rfc', 'cbo', 'loc','amc', 'cam','ce','npm', 'wmc', 'avg_cc', 'lcom', 'ca']]
X_test_scaled_featured = X_test_scaled[['rfc', 'cbo', 'loc','amc', 'cam','ce','npm', 'wmc', 'avg_cc', 'lcom', 'ca']]
X_cv_featured = X_cv[['rfc', 'cbo', 'loc','amc', 'cam','ce','npm', 'wmc', 'avg_cc', 'lcom', 'ca']]

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured, y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured , y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 1, 'gamma': 1, 'kernel': 'linear'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 50, 'max_features': 0.7, 'n_estimators': 500}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 1.0, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df

# Onuncu Datasetin Scale, Feature Selection ve Modelling işlemleri

In [ ]:
# Shuffle data before removing classname to keep mapping
xercesV1 = df[9].sample(frac=1.0)

# We do not need confusing variables such as the class name; We can use it as an index or leave it.
X = xercesV1.iloc[:, :-1]
y = xercesV1["bug"]

print("X:", X.shape)
print("y:", y.shape)

In [ ]:
xercesV1.describe()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Label convert binary 
y = np.where((y > 1), 1, y) 
y = pd.Series(y)

In [ ]:
print([y > 1])

In [ ]:
#Class Balance: we can see bugs rate
unique, counts = np.unique(y, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Splitting data train, test, and cross validation
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_cv, X_test, y_cv, y_test = train_test_split(X_test, y_test, test_size=0.5)
print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

In [ ]:
#we can see y_train bugs rate
unique, counts = np.unique(y_train, return_counts=True)
print("Classes:", unique.tolist())
print("Counts:", counts.tolist())

plt.bar(unique, counts, color=['g', 'orange'], alpha=0.7)
plt.title("#Bugs VS Occurrences")
plt.xticks(range(len(unique)))
plt.ylabel("Occurrences")
plt.xlabel("# Bugs");

In [ ]:
#Scaling features
X_train_scaled = pd.DataFrame(StandardScaler().fit_transform(X_train.values), columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(StandardScaler().fit_transform(X_test.values), columns=X_test.columns, index=X_test.index)

data_for_viz = X_train_scaled.copy()
data_for_viz_unscaled = X_train.copy()

data_for_viz['Bugs'] = y_train.copy().tolist()
data_for_viz_unscaled['Bugs'] = y_train.copy().tolist()

print("Train:", X_train.shape, y_train.shape,
      "Test:", X_test.shape, y_test.shape,
      "Cross Validation", X_cv.shape, y_cv.shape)

Train: (411, 20) (411,) Test: (89, 20) (89,) Cross Validation (88, 20) (88,)


In [ ]:
data_for_viz.head()

In [ ]:
#Correlation matrix
fig, ax = plt.subplots(figsize=(10,8))

corr = data_for_viz.corr()
mask = np.triu(corr)
sns.heatmap(corr, vmin=-1, vmax=1, center= 0, cmap= 'Pastel2', linewidths=3, ax=ax, mask=mask);

#We can see that highest correlated variables with Bugs are:

#1) wmc 
# highest correlated with rfc, npm, loc, ce, dam, 

In [ ]:
#Feature Importance / Selection
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train);
coef_rfc = pd.Series(rfc.feature_importances_, index = X_train.columns)

In [ ]:
preds = rfc.predict(X_train)
print("accuracy score:", accuracy_score(y_train, preds))
#Acc is high and its normal because of only x_train modelling

accuracy score: 0.9975669099756691


In [ ]:
coef_rfc.sort_values()[7:].plot(kind='barh', cmap="Pastel2");

In [ ]:
data_for_viz_unscaled.head()

In [ ]:
for_pair_plot = data_for_viz_unscaled[['ce', 'cbo', 'loc','amc', 'cam', 'Bugs']]
pairplot = sns.pairplot(for_pair_plot, hue="Bugs", vars=['ce', 'cbo', 'loc','amc', 'cam']);
pairplot.fig.suptitle("Scatter w.r.t classes: 0, > 1 Bugs", y=1.02);

In [ ]:
#MODELING
def Modeler(model, X_train, y_train, X_cv, y_cv, params, scale=True, n_jobs=True, pca=False):
    
    if scale & pca:
        pipeline = Pipeline([
            ('scale', StandardScaler()),
            ('dim_reduct', PCA(n_components=4)),
            ('clf', model())])
        print("Scaling and dim reduct...")
        
    elif scale:
        pipeline = Pipeline([ ('scale', StandardScaler()), ('clf', model())])
        
    elif pca:
        pipeline = Pipeline([
        ('dim_reduct', PCA(n_components=pca)),
        ('clf', model())])
    else:
        pipeline = Pipeline([('clf', model())])
        
    grid = GridSearchCV(pipeline, param_grid = params, cv=3, n_jobs=4, refit=True)
    grid.fit(X_cv, y_cv)
    
    best_prameters = {}
    for k, v in grid.best_params_.items():
        best_prameters[k[5:]] = v

    if n_jobs:
        model = model(**best_prameters, n_jobs=4)
    else:
        model = model(**best_prameters)
   
    model.fit(X_train, y_train)
    
    print("Best Parameters for model:", best_prameters)

    return {
        "model": model, 
        "best_params":best_prameters
    }

In [ ]:
def model_eval(model, X_test, y_test, acc=True, auc_=True, plot_conf=True, plot_auc=True, multi=True, f1=True, prop=False):
    
    res = [None, None, None] # Accuracy, auc_roc, f1_score
    y_pred = model.predict(X_test)
    
    if prop:
        y_pred = np.where(y_pred > 0.5, 1, 0)
    
    if f1:
        f_sc = f1_score(y_test, y_pred, average='macro')
        res[2] = f_sc
        
    if acc:
        res[0] = accuracy_score(y_test, y_pred)
#         print('accuracy:', res[0] * 100 ,'%')

    if auc_:
        lb = LabelBinarizer()
        lb.fit(y_test)

        truth = lb.transform(y_test)
        y_pred_encode = lb.transform(y_pred)

        res[1] = roc_auc_score(truth, y_pred_encode, average = 'macro')
#         print('ROC:', res[1] * 100 ,'%')

    if plot_conf:
        fig, ax = plt.subplots(figsize=(10,6))

        conf = confusion_matrix(y_test, y_pred, labels=np.unique(y_test))
        sns.heatmap(conf, cmap= 'Set1', annot=True, cbar=False)
        plt.ylabel('Actual')
        plt.xlabel('Predicted')
        plt.title('Confusion Matrix');
    
    if plot_auc:
        fig, ax = plt.subplots(figsize=(10,6))

        if not auc_:
            lb = LabelBinarizer()
            lb.fit(y_test)

            truth = lb.transform(y_test)
            y_pred_encode = lb.transform(y_pred)

        fpr = [None, None, None]
        tpr = [None, None, None]
        auc_of_roc = [None, None, None]
        
        n_classes = 1 #(0, 1)
        
        for i in range(n_classes):
            fpr[i], tpr[i], _ = roc_curve(truth[:, i], y_pred_encode[:, i])
            auc_of_roc[i] = auc(fpr[i], tpr[i])
            
        plt.plot(fpr[0], tpr[0], label='0 - area under ROC = %0.3f' % auc_of_roc[0])
        if multi:
            plt.plot(fpr[1], tpr[1], label='1 - area under ROC = %0.3f' % auc_of_roc[1])

        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title("ROC AUC");
        plt.legend()

    return res

In [ ]:
X_train_scaled_featured = X_train_scaled[['ce', 'cbo', 'loc','amc', 'cam']]
X_test_scaled_featured = X_test_scaled[['ce', 'cbo', 'loc','amc', 'cam']]
X_cv_featured = X_cv[['ce', 'cbo', 'loc','amc', 'cam']]

In [ ]:
from imblearn.over_sampling import SMOTE
X_train_over_sampled, y_train_over_sampled = SMOTE().fit_resample(X_train_scaled_featured , y_train)
X_cv_over_sampled, y_cv_over_sampled = SMOTE().fit_resample(X_cv_featured, y_cv)

In [ ]:
# print((y_train_over_sampled == 1).sum())
# print((y_train_over_sampled == 0).sum())

# print((y_cv_over_sampled == 1).sum())
# print((y_cv_over_sampled == 0).sum())

X_train_over_sampled = pd.DataFrame(X_train_over_sampled)
X_cv_over_sampled = pd.DataFrame(X_cv_over_sampled)

y_train_over_sampled = pd.Series(y_train_over_sampled)
y_cv_over_sampled = pd.Series(y_cv_over_sampled)

In [ ]:
# SVC()
svc_params = {
    'clf__C': [0.1, 1, 10],
    'clf__kernel': ['linear', 'rbf'],
    'clf__gamma':[1, 0.01, 0.001]
}

svc = Modeler(SVC, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, svc_params, scale=True, n_jobs=False)

Best Parameters for model: {'C': 1, 'gamma': 1, 'kernel': 'linear'}


In [ ]:
model_scoring = model_eval(svc["model"], X_test_scaled_featured, y_test, multi=False)
scores = {}
scores["Support-Vector Machine | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#KNN
knn_params = {
    'clf__n_neighbors': [3, 5, 11, 17], # Try not to put even numbers
    'clf__weights': ['uniform', 'distance'],
    'clf__metric': ['euclidean', 'manhattan'],
}

knn = Modeler(KNeighborsClassifier,X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, knn_params, scale=True)

Best Parameters for model: {'metric': 'euclidean', 'n_neighbors': 11, 'weights': 'distance'}


In [ ]:
model_scoring = model_eval(knn["model"], X_test_scaled_featured, y_test, multi=False)
scores["K-Nearest Neighbor | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#RANDOM FOREST
rfc_params = {
    'clf__n_estimators'      : [200, 500, 1000],
    'clf__max_depth'         : [10, 20, 50],
    'clf__max_features': [1.0, 0.7, 0.4],
    'clf__criterion' :['gini', 'entropy']
}

rfc = Modeler(RandomForestClassifier,X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, rfc_params, scale=True) 

Best Parameters for model: {'criterion': 'gini', 'max_depth': 50, 'max_features': 0.7, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(rfc["model"], X_test_scaled_featured, y_test, multi=False)
scores["Random Forest | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
#ADA BOOST
# AdaBoostClassifier
ada_params = {
    'clf__learning_rate': [0.1, 0.5, 1.0],
    'clf__n_estimators': [100, 200]
}

ada = Modeler(AdaBoostClassifier, X_train_over_sampled, y_train_over_sampled, X_cv_over_sampled, y_cv_over_sampled, ada_params, scale=True, n_jobs=False)

Best Parameters for model: {'learning_rate': 0.5, 'n_estimators': 200}


In [ ]:
model_scoring = model_eval(ada["model"], X_test_scaled_featured, y_test, multi=False)
scores["AdaBoost Classifier | Binary"] = {"Accuracy": model_scoring[0],
                             "ROC": model_scoring[1],
                             "F1-Score": model_scoring[2]}

In [ ]:
score_df = pd.DataFrame.from_dict(scores).T
score_df= score_df.sort_values('ROC', ascending=False)

In [ ]:
score_df